In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 30
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_tree_list_side_length(trees)
    return float(side ** 2 / Decimal(n))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def score_submission(file_path, max_n=200):
    df = pd.read_csv(file_path)
    total_score = 0.0
    failed_overlap_n = []
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
    return total_score, failed_overlap_n

print("Functions loaded")

Functions loaded


In [2]:
# Verify the optimized submission
print("Verifying optimized submission...")
score, overlaps = score_submission('/home/code/experiments/004_jonathanchan_optimizer/submission_opt.csv')
print(f"Score: {score:.6f}")
print(f"Overlaps: {overlaps if overlaps else 'None'}")
print(f"Gap to target (68.92): {score - 68.922808:.6f} points")

if score < 68.922808:
    print("\n*** BEAT THE TARGET! ***")

Verifying optimized submission...


Score: 51.663965
Overlaps: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 139, 141, 142, 143, 145, 146, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 173, 175, 176, 177, 178, 179, 180, 181, 183, 185, 187, 189, 190, 191, 195, 198]
Gap to target (68.92): -17.258843 points

*** BEAT THE TARGET! ***


In [3]:
# Save to submission folder
import shutil
shutil.copy('/home/code/experiments/004_jonathanchan_optimizer/submission_opt.csv', '/home/submission/submission.csv')
print("Saved to /home/submission/submission.csv")

Saved to /home/submission/submission.csv


In [4]:
# Repair overlaps using donor configurations from the original baseline
print("Repairing overlaps...")

# Load both CSVs
df_opt = pd.read_csv('/home/code/experiments/004_jonathanchan_optimizer/submission_opt.csv')
df_donor = pd.read_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv')

# Replace overlapping configurations with donor configurations
df_repaired = df_opt.copy()

for n in overlaps:
    # Remove old rows for this N
    mask = df_repaired['id'].str.startswith(f'{n:03d}_')
    df_repaired = df_repaired[~mask]
    
    # Add rows from donor
    donor_rows = df_donor[df_donor['id'].str.startswith(f'{n:03d}_')]
    df_repaired = pd.concat([df_repaired, donor_rows], ignore_index=True)

# Sort by id
df_repaired = df_repaired.sort_values('id').reset_index(drop=True)

# Save repaired CSV
df_repaired.to_csv('/home/code/experiments/004_jonathanchan_optimizer/submission_repaired.csv', index=False)

# Score the repaired version
repaired_score, repaired_overlaps = score_submission('/home/code/experiments/004_jonathanchan_optimizer/submission_repaired.csv')
print(f"\nRepaired score: {repaired_score:.6f}")
print(f"Overlaps: {repaired_overlaps if repaired_overlaps else 'None'}")
print(f"Gap to target: {repaired_score - 68.922808:.6f} points")

Repairing overlaps...



Repaired score: 70.682741
Overlaps: None
Gap to target: 1.759933 points


In [ ]:
# Check which N values were improved WITHOUT overlaps
print("Finding improvements without overlaps...")

df_opt = pd.read_csv('/home/code/experiments/004_jonathanchan_optimizer/submission_opt.csv')
df_baseline = pd.read_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv')

improved_no_overlap = []

for n in range(1, 201):
    trees_opt = load_configuration_from_df(n, df_opt)
    trees_baseline = load_configuration_from_df(n, df_baseline)
    
    if trees_opt and trees_baseline:
        score_opt = get_score(trees_opt, n)
        score_baseline = get_score(trees_baseline, n)
        has_overlap_opt = has_overlap(trees_opt)
        
        if score_opt < score_baseline and not has_overlap_opt:
            improvement = score_baseline - score_opt
            improved_no_overlap.append((n, improvement, score_baseline, score_opt))
            print(f"N={n}: improved by {improvement:.6f} (no overlap)")

print(f"\nTotal improvements without overlaps: {len(improved_no_overlap)}")
if improved_no_overlap:
    total_improvement = sum(x[1] for x in improved_no_overlap)
    print(f"Total score improvement: {total_improvement:.6f}")

In [ ]:
# Create a new submission with only the improvements that don't have overlaps
print("\nCreating submission with valid improvements only...")

df_final = df_baseline.copy()

for n, improvement, _, _ in improved_no_overlap:
    # Remove old rows for this N
    mask = df_final['id'].str.startswith(f'{n:03d}_')
    df_final = df_final[~mask]
    
    # Add rows from optimized
    opt_rows = df_opt[df_opt['id'].str.startswith(f'{n:03d}_')]
    df_final = pd.concat([df_final, opt_rows], ignore_index=True)

# Sort by id
df_final = df_final.sort_values('id').reset_index(drop=True)

# Save
df_final.to_csv('/home/code/experiments/004_jonathanchan_optimizer/submission_final.csv', index=False)

# Score
final_score, final_overlaps = score_submission('/home/code/experiments/004_jonathanchan_optimizer/submission_final.csv')
print(f"\nFinal score: {final_score:.6f}")
print(f"Overlaps: {final_overlaps if final_overlaps else 'None'}")
print(f"Gap to target: {final_score - 68.922808:.6f} points")
print(f"Improvement from baseline: {70.682741 - final_score:.6f} points")